In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
df = pd.read_csv("./data/pre_processed_credit.csv")
df.head(2)

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_cred_hist_length,loan_to_income_ratio,loan_to_emp_length_ratio,...,medium,large,very large,A,B,C,D,E,F,G
0,21,9600,5.0,1000,11.14,0,0.10,2,0.104167,0.005000,...,0,0,0,0,1,0,0,0,0,0
1,25,9600,1.0,5500,12.87,1,0.57,3,0.572917,0.000182,...,1,0,0,0,0,1,0,0,0,0


In [4]:
df.columns

Index(['person_age', 'person_income', 'person_emp_length', 'loan_amnt',
       'loan_int_rate', 'loan_status', 'loan_percent_income',
       'cb_person_cred_hist_length', 'loan_to_income_ratio',
       'loan_to_emp_length_ratio', 'int_rate_to_loan_amt_ratio', 'MORTGAGE',
       'OTHER', 'OWN', 'RENT', 'DEBTCONSOLIDATION', 'EDUCATION',
       'HOMEIMPROVEMENT', 'MEDICAL', 'PERSONAL', 'VENTURE', 'N', 'Y', '20-25',
       '26-35', '36-45', '46-55', '56-65', 'low', 'low-middle', 'middle',
       'high-middle', 'high', 'small', 'medium', 'large', 'very large', 'A',
       'B', 'C', 'D', 'E', 'F', 'G'],
      dtype='object')

In [5]:
y = df["loan_status"]

In [6]:
df.drop(['loan_status'],  axis =1 ,inplace=True)

In [7]:
df.columns

Index(['person_age', 'person_income', 'person_emp_length', 'loan_amnt',
       'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length',
       'loan_to_income_ratio', 'loan_to_emp_length_ratio',
       'int_rate_to_loan_amt_ratio', 'MORTGAGE', 'OTHER', 'OWN', 'RENT',
       'DEBTCONSOLIDATION', 'EDUCATION', 'HOMEIMPROVEMENT', 'MEDICAL',
       'PERSONAL', 'VENTURE', 'N', 'Y', '20-25', '26-35', '36-45', '46-55',
       '56-65', 'low', 'low-middle', 'middle', 'high-middle', 'high', 'small',
       'medium', 'large', 'very large', 'A', 'B', 'C', 'D', 'E', 'F', 'G'],
      dtype='object')

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.33, random_state=42)

In [9]:
X_train.shape

(19182, 43)

In [10]:
X_test.shape

(9448, 43)

In [11]:
y_train.shape

(19182,)

In [12]:
y_test.shape

(9448,)

# Normalize data

In [13]:
# data normalization with sklearn
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(X_train)

# transform training data
X_train_norm = norm.transform(X_train)

# transform testing dataabs
X_test_norm = norm.transform(X_test)

In [14]:
X_train = X_train_norm
X_test = X_test_norm

# LOGISTIC REGRESSION

In [15]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(penalty= 'l1', solver='saga')

In [16]:
param_grid = [    
    {#'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : np.logspace(-4, 4, 20)
    }
]

In [17]:
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(LR, param_grid = param_grid, cv = 3, verbose=True, n_jobs=-1)

In [18]:
best_clf = clf.fit(X_train,y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


In [19]:
best_clf.best_estimator_

LogisticRegression(C=3792.690190732246, penalty='l1', solver='saga')

In [20]:
print (f'Accuracy: {best_clf.score(X_train,y_train):.3f}')

Accuracy: 0.879


In [21]:
y_pred_LR = best_clf.predict(X_test)

In [22]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred_LR)

array([[7016,  349],
       [ 859, 1224]], dtype=int64)

In [23]:
from sklearn.metrics import classification_report
print("Classification report - \n", classification_report(y_test,y_pred_LR))

Classification report - 
               precision    recall  f1-score   support

           0       0.89      0.95      0.92      7365
           1       0.78      0.59      0.67      2083

    accuracy                           0.87      9448
   macro avg       0.83      0.77      0.80      9448
weighted avg       0.87      0.87      0.87      9448



### penalty= 'l2', solver='saga'

In [24]:
LR = LogisticRegression(penalty= 'l2', solver='liblinear')
LR.fit(X_train,y_train)

LogisticRegression(solver='liblinear')

In [25]:
y_pred_LR = LR.predict(X_test)

In [26]:
confusion_matrix(y_test,y_pred_LR)

array([[7016,  349],
       [ 890, 1193]], dtype=int64)

In [27]:
print("Classification report - \n", classification_report(y_test,y_pred_LR))

Classification report - 
               precision    recall  f1-score   support

           0       0.89      0.95      0.92      7365
           1       0.77      0.57      0.66      2083

    accuracy                           0.87      9448
   macro avg       0.83      0.76      0.79      9448
weighted avg       0.86      0.87      0.86      9448



In [28]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred_LR)

0.868861134631668

# Decision Tree

In [29]:
from sklearn.tree import DecisionTreeClassifier#for checking testing results
from sklearn.metrics import classification_report, confusion_matrix#for visualizing tree 

In [30]:
# Defining the decision tree algorithm
dtree=DecisionTreeClassifier(max_depth=100)
dtree.fit(X_train,y_train)
print('Decision Tree Classifier Created')

Decision Tree Classifier Created


In [31]:
# Predicting the values of test data
y_pred_DT = dtree.predict(X_test)
print("Classification report - \n", classification_report(y_test,y_pred_DT))

Classification report - 
               precision    recall  f1-score   support

           0       0.93      0.92      0.93      7365
           1       0.73      0.77      0.75      2083

    accuracy                           0.89      9448
   macro avg       0.83      0.84      0.84      9448
weighted avg       0.89      0.89      0.89      9448



In [32]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred_DT)

array([[6775,  590],
       [ 486, 1597]], dtype=int64)

In [33]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred_DT)

0.8861134631668078

# RANDOM FOREST

In [34]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(n_estimators=50)

#RF.fit(X_train_scaled,y_train)
RF.fit(X_train,y_train)

RandomForestClassifier(n_estimators=50)

In [35]:
#y_pred_RF = RF.predict(X_test_scaled)
y_pred_RF = RF.predict(X_test)

In [36]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred_RF)

array([[7305,   60],
       [ 589, 1494]], dtype=int64)

In [37]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred_RF)

0.9313082133784928

In [38]:
print("Classification report - \n", classification_report(y_test,y_pred_RF))

Classification report - 
               precision    recall  f1-score   support

           0       0.93      0.99      0.96      7365
           1       0.96      0.72      0.82      2083

    accuracy                           0.93      9448
   macro avg       0.94      0.85      0.89      9448
weighted avg       0.93      0.93      0.93      9448



# Neural Network

In [39]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import keras

In [40]:
# define the keras model
import tensorflow as tf

optimizer = tf.keras.optimizers.Adam(lr=0.001)
model = Sequential()
model.add(Dense(32, input_dim=43, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.fit(X_train, y_train, epochs=200, batch_size=1000)
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/200


C:\Users\My Pc\AppData\Roaming\Python\Python38\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


20/20 [==============================] - 1s 4ms/step - loss: 0.6346 - accuracy: 0.6961
Epoch 2/200
20/20 [==============================] - 0s 4ms/step - loss: 0.5535 - accuracy: 0.7754
Epoch 3/200
20/20 [==============================] - 0s 4ms/step - loss: 0.5118 - accuracy: 0.7826
Epoch 4/200
20/20 [==============================] - 0s 3ms/step - loss: 0.4789 - accuracy: 0.7856
Epoch 5/200
20/20 [==============================] - 0s 3ms/step - loss: 0.4499 - accuracy: 0.7904
Epoch 6/200
20/20 [==============================] - 0s 3ms/step - loss: 0.4253 - accuracy: 0.8027
Epoch 7/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4113 - accuracy: 0.8225
Epoch 8/200
20/20 [==============================] - 0s 3ms/step - loss: 0.3946 - accuracy: 0.8370
Epoch 9/200
20/20 [==============================] - 0s 3ms/step - loss: 0.3830 - accuracy: 0.8486
Epoch 10/200
20/20 [==============================] - 0s 3ms/step - loss: 0.3725 - accuracy: 0.8520
Epoch 11/200
20/20 [=

20/20 [==============================] - 0s 2ms/step - loss: 0.2548 - accuracy: 0.9127
Epoch 84/200
20/20 [==============================] - 0s 2ms/step - loss: 0.2568 - accuracy: 0.9136
Epoch 85/200
20/20 [==============================] - 0s 2ms/step - loss: 0.2583 - accuracy: 0.9118
Epoch 86/200
20/20 [==============================] - 0s 2ms/step - loss: 0.2566 - accuracy: 0.9123
Epoch 87/200
20/20 [==============================] - 0s 2ms/step - loss: 0.2573 - accuracy: 0.9112
Epoch 88/200
20/20 [==============================] - 0s 3ms/step - loss: 0.2586 - accuracy: 0.9127
Epoch 89/200
20/20 [==============================] - 0s 2ms/step - loss: 0.2562 - accuracy: 0.9141
Epoch 90/200
20/20 [==============================] - 0s 2ms/step - loss: 0.2558 - accuracy: 0.9130
Epoch 91/200
20/20 [==============================] - 0s 2ms/step - loss: 0.2567 - accuracy: 0.9125
Epoch 92/200
20/20 [==============================] - 0s 2ms/step - loss: 0.2556 - accuracy: 0.9143
Epoch 93/200


20/20 [==============================] - 0s 2ms/step - loss: 0.2382 - accuracy: 0.9204
Epoch 165/200
20/20 [==============================] - 0s 2ms/step - loss: 0.2386 - accuracy: 0.9204
Epoch 166/200
20/20 [==============================] - 0s 2ms/step - loss: 0.2375 - accuracy: 0.9204
Epoch 167/200
20/20 [==============================] - 0s 2ms/step - loss: 0.2385 - accuracy: 0.9196
Epoch 168/200
20/20 [==============================] - 0s 2ms/step - loss: 0.2377 - accuracy: 0.9191
Epoch 169/200
20/20 [==============================] - 0s 2ms/step - loss: 0.2393 - accuracy: 0.9181
Epoch 170/200
20/20 [==============================] - 0s 2ms/step - loss: 0.2376 - accuracy: 0.9199
Epoch 171/200
20/20 [==============================] - 0s 3ms/step - loss: 0.2362 - accuracy: 0.9211
Epoch 172/200
20/20 [==============================] - 0s 3ms/step - loss: 0.2374 - accuracy: 0.9202
Epoch 173/200
20/20 [==============================] - 0s 3ms/step - loss: 0.2396 - accuracy: 0.9199
Epoc

In [41]:
loss, acc = model.evaluate(X_test, y_test, verbose=0)

In [42]:
loss

0.23683565855026245

In [43]:
acc

0.9185012578964233

In [44]:
predict_x=model.predict(X_test) 
yhat_classes=np.argmax(predict_x,axis=1)

In [45]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, yhat_classes)
print("Accuracy: %f" % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, yhat_classes, average='weighted', labels=np.unique(yhat_classes))
print("Precision: %f" % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, yhat_classes, average='weighted', labels=np.unique(yhat_classes))
print("Recall: %f" % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, yhat_classes, average='weighted', labels=np.unique(yhat_classes))
print("F1 score: %f" % f1)

# confusion matrix
matrix = confusion_matrix(y_test, yhat_classes)
print(matrix)

Accuracy: 0.779530
Precision: 0.779530
Recall: 1.000000
F1 score: 0.876108
[[7365    0]
 [2083    0]]
